In [20]:
import os, pandas as pd, numpy as np, re, time
from functools import reduce
pd.set_option('display.max_columns', 1500)

#def main():
sage = '/home/wraikes/Dropbox/partnership/dmt/data/sage_not_final'
#sage = r'C:\Users\williamr.PDFK\Dropbox\partnership\dmt_temp\Sage (Mobile Phone) Data 8.31.17'
os.chdir(sage)
files_to_exclude = ['digital-marshmallow-status_8.31.17.csv',
                    'digital-marshmallow-appVersion_8.31.17.csv']
end_string = '_8.31.17.csv'

In [35]:
def main():
    sage = '/home/wraikes/Dropbox/partnership/dmt/data/sage_not_final'
    #sage = r'C:\Users\williamr.PDFK\Dropbox\partnership\dmt_temp\Sage (Mobile Phone) Data 8.31.17'
    os.chdir(sage)
    
    files_to_exclude = ['digital-marshmallow-status_8.31.17.csv',
                        'digital-marshmallow-appVersion_8.31.17.csv']
    end_string = '_8.31.17.csv'
    bl = 'baseline'
    _21 = '21-day-assessment'
    
    common_df = clean_common_dfs(os.listdir(), files_to_exclude, end_string)
    
    #To target the behavior_choices_1 df
    common_df = selected_recode(common_df)
        
    bart_att = [
        [bl, 'BART0.25'],
        [bl, 'BART250.00'],
        [_21, 'BART0.25'],
        [_21, 'BART250.00']       
    ]
    
    bart_df = clean_df('digital-marshmallow-bart-v4_8.31.17.csv',
                   end_string,
                   bart_att,
                   var_2='data.json.variable_label')
    
    delay_att = [
        [bl, 'dd_time_6_months'],
        [bl, 'dd_money_6_month'],
        [bl, 'dd_money_1_month'],
        [bl, 'dd_time_1_year'],
        [_21, 'dd_time_6_month'],
        [_21, 'dd_money_6_month'],
        [_21, 'dd_money_1_month'],
        [_21, 'dd_time_1_year']            
    ]
    
    delay_df = clean_df('digital-marshmallow-delay_discounting_raw-v6_8.31.17.csv',
                    end_string,
                    delay_att,
                    var_2='data.json.variableLabel')    
    
    delay_df = df_array_last_value(delay_df)
    delay_df = df_split_array(delay_df)
    
    behave_4_att = [
        [bl]
    ]
    
    behave_4_df = clean_df('digital-marshmallow-behavior_choices_4_bl-v2_8.31.17.csv',
                       end_string,
                       behave_4_att)
    
    behave_4_df = selected_spread(behave_4_df)
    
    discount_att = [
        [bl, 'pd_constant_money'],
        [bl, 'pd_constant_probabiliy'],
        [_21, 'pd_constant_money'],
        [_21, 'pd_constant_probability']
    ]
    
    discount_df = clean_df('digital-marshmallow-discounting_raw-v2_8.31.17.csv',
                      end_string,
                      discount_att,
                      var_2='data.json.variableLabel')
    
    evening_note_att = [
        [bl]
    ] 
    
    evening_note_df = clean_df('digital-marshmallow-evening_notification_time-v2_8.31.17.csv', 
                           end_string,
                           evening_note_att) 
    
    gonogo_att = [
        [bl, 'go_no_go_stable_stimulus_active_task'],
        [bl, 'go_no_go_variable_stimulus_active_task'],
        [_21, 'go_no_go_stable_stimulus_active_task'],
        [_21, 'go_no_go_variable_stimulus_active_task']
    ]
    
    gonogo_df = clean_df('digital-marshmallow-goNoGo-v2_8.31.17.csv', 
                     end_string,
                     gonogo_att,
                     var_2='data.json.variable_label')
    
    morning_note_att = [
        [bl]
    ]   
    
    morning_note_df = clean_df('digital-marshmallow-morning_notification_time-v3_8.31.17.csv',
                          end_string,
                          morning_note_att)
    
    pam_mult_att = [
        [bl],
        [_21]
    ]
     
    pam_mult_df = clean_df('digital-marshmallow-pam_multiple-v2_8.31.17.csv',
                      end_string,
                      pam_mult_att)
    
    df = merge_dfs([common_df, 
                        bart_df,
                        delay_df,
                        behave_4_df,
                        discount_df,
                        evening_note_df,
                        gonogo_df,
                        morning_note_df,
                        pam_mult_df])  

    df = timestamp_conv(df)
    
    return df

In [22]:
def merge_dfs(dfs):
    df = reduce(lambda left, right: pd.merge(left, right, how = 'outer',
                                             on='externalId'), 
                dfs)
    
    return df

In [23]:
def clean_common_dfs(directory, exclude_files, string):
    '''
    Go through and process all files in directory that don't need columns "spread".
    If extra processing is needed, a different function will be used.
    Return a single dataframe, of all data merged.
    '''
    
    new_dfs = []
    
    for file in directory: 
        if file not in exclude_files:
            df_name = create_df_name(file, string)
            df = pd.read_csv(file)
            df = remove_dupes(df)
            if df_name in ('demographics-v2', 'generally_sem_diff_bl-v2'):
                df = clean_df_dupe(df, 'yQ7pYy')
            
            if len(df.externalId) == len(df.externalId.unique()):
                df = col_relabel(df, df_name)
                new_dfs.append(df)
    
    return merge_dfs(new_dfs)

In [24]:
def create_df_name(string, end_string):
    string = string.replace('digital-marshmallow-', '')
    string = string.replace(end_string, '')
    
    return string 

In [25]:
def col_relabel(df, prefix, att=None):
    '''
    Remove: 'metadata.json.' and 'data.json.'.
    Replace spaces with underscores.
    Append new name to existing column name, except if externalId.
    '''
    new_cols = []
    col_re = re.compile('metadata.json.|data.json.')
    
    for col in df.columns:
        new_col = re.sub(col_re, '', col)
    
        if new_col != 'externalId':
            if att and len(att) == 2:
                new_col = 'SAGE_{}_{}_{}___{}'.format(prefix, 
                                                      att[0], 
                                                      att[1], 
                                                      new_col
                                                     )
            elif att and len(att) == 1:
                new_col = 'SAGE_{}_{}___{}'.format(prefix, 
                                                   att[0], 
                                                   new_col
                                                  )                
            
            else:
                new_col = 'SAGE_{}___{}'.format(prefix, 
                                                new_col
                                                ) 
                                              
        new_col = new_col.replace(' ', '_')
        new_col = new_col.replace('-', '_')
        new_cols.append(new_col)
    
    df.columns = new_cols
    
    return df

In [26]:
def remove_dupes(df):
    
    test_users = [
        'ThpMV2Achc', 
        'SEkQVTCe6j', 
        'Wh8NSX3DHL', 
        'SaXFr2kPZa', 
        'VWUcSp4TeH', 
        'yXEfAmW682', 
        'gwEpQR8j9B',
        'WbbNWM4RAF', 
        'D5bzYrfd8E', 
        'LJcmEFWp74', 
        'ULoF3MM1nN',
        'G9Bbsg'
    ]

    diff_study = [
        'rL8eA3',
        'rLg5xs',
        'rLrD9h',
        'rLP7H2',
        'rL6s6h',
        'aOyzBg',
        'aORA43',
        'aOh48U',
        'aOLu4K',
        'aOQtxv',
        'aO5TvQ',
        'mPC9S8',
        'mPgquX',
        'mP5xkB',
        'mPSQvh',
        'mPYk2p',
        'mP3rbd'
    ]

    
    return df[~df.externalId.isin(test_users + diff_study)]

In [27]:
def spread_cols(df, df_name, attributes, var_2):
    '''
    Merge the dataframes into one, with new columns.
    '''
    dfs = []
    var_1 = 'metadata.json.taskIdentifier'
    
    for att in attributes:
        cols = df.columns
        new_df = pd.DataFrame(columns=cols)
        
        for ix, row in df.iterrows():
            if var_2:
                if row[var_1] == att[0] and row[var_2] == att[1]:
                    new_df = new_df.append(row, ignore_index=True)
            else:
                if row[var_1] == att[0]:
                    new_df = new_df.append(row, ignore_index=True)
                    
        new_df = col_relabel(new_df, df_name, att)
        dfs.append(new_df)
    
    return merge_dfs(dfs)

In [28]:
def clean_df(file, string, attributes, var_2=None):
    df_name, df = create_df_name(file, string), pd.read_csv(file)
    df = remove_dupes(df)
    
    if df_name == 'goNoGo-v2':
        df = clean_df_dupe(df, 'ksJM3Y')
    
    if df_name == 'behavior_choices_4_bl-v2':
        df = clean_df_dupe(df, 'yQ7pYy')
    
    df = spread_cols(df, df_name, attributes, var_2)
    
    return df

In [29]:
def clean_df_dupe(df, _id):
   
    indices = df.index[df.externalId == _id]
    df.drop(indices[-1], axis=0, inplace=True)
    
    return df

In [30]:
def timestamp_conv(df):
    
    for col in df.columns:
        if any([col.endswith(x) for x in ('createdOn', 'startDate', 'endDate')]):
            df[col] = df[col][df[col].notnull()].apply(lambda x: time.strftime("%m.%d.%Y %H:%M - %Z", time.localtime(x / 1000)))
    
    return df

In [31]:
def array_split(value, i):
    if pd.notnull(value):
        return list(map(float, value[1:-1].split(',')))[i]
    else:
        return value

def df_array_last_value(df):
    new_df = df.copy()

    for col in df.columns:
        if 'nowArray' in col:
            new_col = col + '_last'
            new_df[new_col] = df[col].apply(lambda x: array_split(x, -1))
    
    return new_df

def df_split_array(df):
    new_df = df.copy()
    
    for col in df.columns:
        if all([x in col for x in ('times', 'delay', 'SAGE')]):
            for i in range(6):
                new_col = col + '_v' + str(i)
                new_df[new_col] = df[col].apply(lambda x: array_split(x, i))
                
    return new_df

In [32]:
def selected_spread(df):
    #Note: codebook is incorrect. 'gamble' vs. 'gambling'.
    terms = ['junkfood', 'gambling', 'smoke', 'mobile', 'overeat', 'mast', 'cut', 'smedia',
             'alcohol', 'porn', 'hair', 'gaming', 'sitting', 'spend', 'temper', 'othermore']

    for term in terms:
        col_name = 'SAGE_behavior_choices_4_bl_v2_baseline___selected' + '_' + term
        df[col_name] = np.where(df['SAGE_behavior_choices_4_bl_v2_baseline___selected'].str.contains(term), 1, 0)
        
    return df

def recode(value):
    
    terms = ['junkfood', 'gambling', 'smoke', 'mobile', 'overeat', 'mast', 'cut', 'smedia',
             'alcohol', 'porn', 'hair', 'gaming', 'sitting', 'spend', 'temper', 'othermore']
    
    for ix, term in enumerate(terms):
        if term in value:
            return ix 

def selected_recode(df):
    
    df['SAGE_behavior_choices_1_bl_v1___selected_recode'] = df['SAGE_behavior_choices_1_bl_v1___selected'].apply(recode)
    
    return df

In [36]:
test = main()

In [37]:
test.head()

SAGE_generally_sem_diff_bl_v2___ROW_ID  \
0                                       6   
1                                       7   
2                                       8   
3                                       9   
4                                      10   

   SAGE_generally_sem_diff_bl_v2___ROW_VERSION  \
0                                            4   
1                                            4   
2                                            5   
3                                            5   
4                                            5   

  SAGE_generally_sem_diff_bl_v2___recordId  \
0     69a8bf0d-0925-410a-ba80-d229c5805fb2   
1     7368d235-9e27-446c-965d-3f43d9708175   
2     30c67484-f34f-4acc-b909-d4d72136e65c   
3     b98f2a97-7c58-437e-8a71-596e21421665   
4     7c261654-dfef-4b3c-981e-3eecb9d17fe1   

  SAGE_generally_sem_diff_bl_v2___appVersion  \
0                      version 1.0, build 16   
1                      version 1.0, build 16   
2                     version 1.0.2, build 1   
3                      version 1.0, build 15   
4                      version 1.0, build 16   

  SAGE_generally_sem_diff_bl_v2___phoneInfo  \
0                      iPhone 6; iOS/10.3.2   
1                     iPhone 7+; iOS/10.3.2   
2                               LGE LGLS775   
3                     iPhone 5S; iOS/10.2.1   
4                      iPhone 7; iOS/10.3.1   

  SAGE_generally_sem_diff_bl_v2___uploadDate  \
0                                   6/6/2017   
1                                   6/6/2017   
2                                   6/7/2017   
3                                   6/7/2017   
4                                   6/7/2017   

  SAGE_generally_sem_diff_bl_v2___healthCode externalId  \
0       2d139622-632a-412d-9021-a6aa4edf6336     GpRu74   
1       c5254f3c-71f3-484f-8827-5b22c0f862d7     Dwr9Wq   
2       a260a30a-eb9f-46f6-88f9-0db97cc01940     up3ALb   
3       8a9ad996-0f1c-407d-93d6-7b8d51a428e5     gSWn9N   
4       f20051f9-c293-4da9-bf7d-453228685657     Uusd4p   

  SAGE_generally_sem_diff_bl_v2___dataGroups  \
0                                        NaN   
1                                        NaN   
2                                        NaN   
3                                  test_user   
4                                        NaN   

  SAGE_generally_sem_diff_bl_v2___createdOn  \
0                    06.05.2017 15:13 - EDT   
1                    06.05.2017 15:13 - EDT   
2                    06.06.2017 19:00 - EDT   
3                    06.06.2017 21:46 - EDT   
4                    06.06.2017 21:46 - EDT   

   SAGE_generally_sem_diff_bl_v2___createdOnTimeZone  \
0                                               -400   
1                                               -400   
2                                               -400   
3                                               -400   
4                                               -400   

  SAGE_generally_sem_diff_bl_v2___userSharingScope  \
0                        ALL_QUALIFIED_RESEARCHERS   
1                        ALL_QUALIFIED_RESEARCHERS   
2                        ALL_QUALIFIED_RESEARCHERS   
3                        ALL_QUALIFIED_RESEARCHERS   
4                        ALL_QUALIFIED_RESEARCHERS   

   SAGE_generally_sem_diff_bl_v2___UUID  \
0  F940E293-C7DC-4E28-9628-D7F92D3972E7   
1  55552D10-62D3-4699-A8F3-8A22EA59DA2C   
2  e7c5829d-af28-4683-9c5f-5567ce2dee33   
3  E772BFB8-E290-4300-A158-C099FBB331ED   
4  D9E1D4C3-0B16-40FD-9BF8-786EE79FB583   

  SAGE_generally_sem_diff_bl_v2___taskIdentifier  \
0                                       baseline   
1                                       baseline   
2                                       baseline   
3                                       baseline   
4                                       baseline   

  SAGE_generally_sem_diff_bl_v2___taskRunUUID  \
0        9AEDD7BF-73E8-4C87-ADFC-1D971E846C48   
1        A73252BC-9D09-4595-B871-0